# 16. Способы сохранения и загрузки моделей в Keras

In [ ]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.datasets import cifar10, mnist
#from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.random.set_seed(1)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#стандартизация данных
x_train = x_train.reshape(-1, 784) / 255
x_test = x_test.reshape(-1, 784) / 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [3]:
model = keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 708us/step - acc: 0.8798 - loss: 0.4218
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 659us/step - acc: 0.9640 - loss: 0.1210
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 672us/step - acc: 0.9772 - loss: 0.0797
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 673us/step - acc: 0.9843 - loss: 0.0581
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 659us/step - acc: 0.9876 - loss: 0.0446


In [ ]:
model.save('sample_model.keras')                                #сохранение модели
model_loaded = keras.models.load_model('sample_model.keras')    #загрузка модели

In [6]:
model_loaded.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - acc: 0.9747 - loss: 0.0840


[0.07412654161453247, 0.9785000085830688]

In [38]:
class NeuralNetwork(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.model_layers = [layers.Dense(n, activation='relu') for n in self.units]
        
    def call(self, inputs):
        x = inputs
        for layer in self.model_layers:
            x = layer(x)
        return x
    
    def get_config(self):
        return {'units': self.units}
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [39]:
class NeuralNetworkLinear(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.model_layers = [layers.Dense(n, activation='linear') for n in self.units]
        
    def call(self, inputs):
        x = inputs
        for layer in self.model_layers:
            x = layer(x)
        return x
    
    def get_config(self):
        return {'units': self.units}
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [48]:
model2 = NeuralNetwork([128, 10])
model3 = NeuralNetwork([128, 10])

y = model2.predict(tf.expand_dims(x_test[0], axis=0))
print(y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[[0.6142287  0.1803405  0.11545718 0.         0.17410582 0.
  0.23236892 0.19918588 0.         0.        ]]


In [46]:
model2.save('16_model.keras')
model_loaded = keras.models.load_model('16_model.keras', custom_objects={'NeuralNetwork' : NeuralNetworkLinear}) 

In [47]:
y = model_loaded.predict(tf.expand_dims(x_test[0], axis=0))
print(y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[ 0.16030347  0.11778446  0.42181933  0.8536183  -0.43894067 -0.24337035
   0.22057478 -0.42201602 -0.28741643 -0.03323155]]


In [51]:
y = model2.predict(tf.expand_dims(x_test[0], axis=0))
print(y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.6142287  0.1803405  0.11545718 0.         0.17410582 0.
  0.23236892 0.19918588 0.         0.        ]]


In [52]:
y = model3.predict(tf.expand_dims(x_test[0], axis=0))
print(y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[[0.08028796 0.32204095 0.         0.         0.         0.
  0.         0.         0.         0.        ]]


In [ ]:
#считывание и запись весовых коэффициентов модели
weights = model2.get_weights()
model3.set_weights(weights)

y = model3.predict(tf.expand_dims(x_test[0], axis=0))
print(y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[0.6142287  0.1803405  0.11545718 0.         0.17410582 0.
  0.23236892 0.19918588 0.         0.        ]]


In [59]:
#сохранение весовых коэф на диск

weights = model2.save_weights('model_weights.weights.h5')
model3.load_weights('model_weights.weights.h5')

y = model3.predict(tf.expand_dims(x_test[0], axis=0))
print(y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[0.6142287  0.1803405  0.11545718 0.         0.17410582 0.
  0.23236892 0.19918588 0.         0.        ]]
